In [1]:
# If you run this notebook on Google Colaboratory, uncomment the below to install automl_alex.
#!pip install --quiet automl_alex

In [1]:
import automl_alex
import sklearn
import time
from automl_alex import AutoML, AutoMLClassifier
print(automl_alex.__version__)

0.05.15


In [2]:
RANDOM_SEED = 42

# Classifier

## Data

In [3]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
dataset = fetch_openml(name='credit-g', version=1, as_frame=True)
dataset.target = dataset.target.astype('category').cat.codes
dataset.data.head(5)

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,4.0,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,2.0,real estate,22.0,none,own,1.0,skilled,1.0,none,yes
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,3.0,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,4.0,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,4.0,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes


In [4]:
X_train, X_test, y_train, y_test = train_test_split(dataset.data, 
                                                    dataset.target,
                                                    test_size=0.25, 
                                                    random_state=RANDOM_SEED,)
X_train.shape, X_test.shape

((750, 20), (250, 20))

## Model

In [5]:
model = AutoML(X_train, y_train, X_test, type_of_estimator='classifier', random_state=RANDOM_SEED)

# or Simply
model = AutoMLClassifier(X_train, y_train, X_test, random_state=RANDOM_SEED)

In [6]:
%%time
predict_test, predict_train = model.fit_predict(timeout=2000, verbose=2)


 Opt StackingModels
One iteration takes ~ 0.5 sec
Start Auto calibration parameters
Start optimization with the parameters:
Score_folds =3
Opt_lvl =3
Cold_start =87.0
Early_stoping =174.0
Metric =roc_auc_score
Direction =maximize
Default model roc_auc_score = 0.6891
########################################
Optimize: : 185it [12:28,  6.07s/it, CatBoost Best Score roc_auc_score = 0.8301 +- 0.041296]
 EarlyStopping Exceeded: Best Score: 0.7888roc_auc_score
Optimize: : 185it [12:28,  4.05s/it, CatBoost Best Score roc_auc_score = 0.8301 +- 0.041296]

 Predict from StackingModels
  0%|          | 0/25 [00:00<?, ?it/s]
 Mean Score roc_auc_score on 30 Folds: 0.7983 std: 0.050993
  4%|▍         | 1/25 [00:47<18:48, 47.02s/it]
 Mean Score roc_auc_score on 30 Folds: 0.8013 std: 0.048718
  8%|▊         | 2/25 [01:32<17:48, 46.44s/it]
 Mean Score roc_auc_score on 30 Folds: 0.8 std: 0.048258
 12%|█▏        | 3/25 [01:54<14:22, 39.19s/it]
 Mean Score roc_auc_score on 30 Folds: 0.7994 std: 0.051537
 

In [7]:
predict_test[:5]

array([0.42217891, 0.48217917, 0.5430897 , 0.34699358, 0.2460748 ])

In [8]:
print('Test AUC: ', round(sklearn.metrics.roc_auc_score(y_test, predict_test),4))

Test AUC:0.7962
